# ETL (Extract, Transform, Load)

In [2]:
# importar bibliotecas:

import chardet
import pandas as pd
import ast
import json

## **1- ORIGEN DE LOS DATOS**

**Observaciones y conclusiones:**

- Se encuentran 3 datasets en archivos diferentes cada uno en formato json pero se encuentran comprimidos así que se procede a descomprimirlos primero con "winrar" antes de importarlos.

- Una vez descomprimirdos se revisa el formato de codificación de cada archivo y se evidencia lo siguiente:
    - australian_user_reviews.json = encoding MacRoman
    - australian_user_items.json = encoding MacRoman
    - output_steam_games.json = encoding ASCII <br>
<br>
- Después de leer cada archivo JSON en un dataframe y dado que toma bastante tiempo realizar este proceso se guarda cada dataframe en formato CSV como respaldo para poder cargarlo de manera más rápida en caso de ser necesario.

### **1-1- LEER DATOS**

In [ ]:
# Examinar el encoding del archivo "australian_user_reviews.json"

def detect_encoding(file_path):
    '''Funcion para detectar la codificacion de un archivo.
        Recibe la ruta del archivo como parámetro'''
    
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

In [ ]:
# Conocer la codificación del archivo "australian_user_reviews.json" para importar su contenido

file_path = 'PI-MLOps-STEAM/australian_user_reviews.json'
encoding = detect_encoding(file_path)

print(f"La codificación del archivo es: {encoding}")

In [ ]:
# Leer datos de "australian_user_reviews.json" e importar contenido en dataframe

rows = []

with open ('PI-MLOps-STEAM/australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rows)
df_reviews.head()

In [ ]:
# Guardar el df con la información de "australian_user_reviews.json" en formato CSV
df_reviews.to_csv('csv_reviews.csv', index=False)

In [ ]:
# Leer datos de "australian_users_items.json" e importar contenido en dataframe

rows = []

with open ('PI-MLOps-STEAM/australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_items = pd.DataFrame(rows)
df_items.head()

In [ ]:
# Guardar el df con la información de "australian_users_items.json" en formato CSV
df_items.to_csv('csv_items.csv', index=False)

In [ ]:
# Conocer la codificación del archivo "output_steam_games.json" para importar su contenido

file_path = 'PI-MLOps-STEAM/output_steam_games.json'
encoding = detect_encoding(file_path)
print(f"La codificación del archivo es: {encoding}")

In [ ]:
# Leer datos de "output_steam_games.json" e importar contenido en dataframe
df_games = pd.read_json('PI-MLOps-STEAM/output_steam_games.json', lines=True)
df_games

In [ ]:
# Guardar el df con la información de "output_steam_games.json" en formato CSV
df_games.to_csv('csv_games.csv', index=False)

### **1-2- ESTRUCTURA GENERAL DE LOS DATASETS**

In [9]:
# Revisar el tipo de datos contenido dentro de cada columna

def verificar_tipo_datos(df):
    '''
    Función para revisar el tipo de datos contenido dentro de cada columna así como el porcentaje de nulos y la cantidad de filas que estan toalmente nulas.
    Recibe como parámetro el dataframe a examinar.
    '''

    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))

    df_info = pd.DataFrame(mi_dict)

    for columna in df.columns:
        print(columna, " (nulos) = ", df[columna].isnull().sum())
    
    print("\nfilas completamente nulas: ", df.isna().all(axis=1).sum())
    
    return df_info

#### **1-2- csv_games.csv = output_steam_games.json**

- Dimensiones y tipos de variables

**Observaciones y conclusiones:**

- Se revisa dimensiones, cantidad de columnas, estructura general y se encuentran 88310 filas (registros) completamente nulos así que se eliminan para reducir el tamaño del arhivo, mejorar el procesamiento y limpiar un poco más los datos.
- El dataset queda listo para exportar a CSV para realizar el "Analisis Exploratorio de Datos" (EDA)

In [10]:
# Lectura de los CSV en un df respectivamente

ruta = "csv_games.csv"
df_games_copy = pd.read_csv(ruta, low_memory=False)

In [11]:
# Ver dimensiones
df_games_copy.shape

(120445, 13)

In [12]:
# Revisar tipos de columnas y nulos
df_games_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [13]:
# Revisar estructura general
df_games_copy.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Revisar los tipos de datos de cada columna y el porcentaje de nulos
verificar_tipo_datos(df_games_copy)

publisher  (nulos) =  96381
genres  (nulos) =  91593
app_name  (nulos) =  88312
title  (nulos) =  90360
url  (nulos) =  88310
release_date  (nulos) =  90377
tags  (nulos) =  88473
reviews_url  (nulos) =  88312
specs  (nulos) =  88980
price  (nulos) =  89687
early_access  (nulos) =  88310
id  (nulos) =  88312
developer  (nulos) =  91609

filas completamente nulas:  88310


,nombre_campo,tipo_datos,no_nulos_%,nulos_%
0,publisher,"[<class 'float'>, <class 'str'>]",19.98,80.02
1,genres,"[<class 'float'>, <class 'str'>]",23.95,76.05
2,app_name,"[<class 'float'>, <class 'str'>]",26.68,73.32
3,title,"[<class 'float'>, <class 'str'>]",24.98,75.02
4,url,"[<class 'float'>, <class 'str'>]",26.68,73.32
5,release_date,"[<class 'float'>, <class 'str'>]",24.96,75.04
6,tags,"[<class 'float'>, <class 'str'>]",26.54,73.46
7,reviews_url,"[<class 'float'>, <class 'str'>]",26.68,73.32
8,specs,"[<class 'float'>, <class 'str'>]",26.12,73.88
9,price,"[<class 'float'>, <class 'str'>]",25.54,74.46


In [15]:
# Eliminar filas donde todas las columnas sean nulas
df_games_copy.dropna(how='all', inplace=True)

#### **1-2- csv_reviews.csv = australian_user_reviews.json**

- Dimensiones y tipos de variables

**Observaciones y conclusiones:**

- Se revisa dimensiones, cantidad de columnas, estructura general y no se encuentran filas completamente nulas.
- Dado que el dataset contiene una columna llamada "reviews" que contiene datos en formato lista compuestas a su vez de diccionarios de procede a desanidar y para dejar un dataset completo 
- El dataset queda listo para exportar a CSV para realizar el "Analisis Exploratorio de Datos" (EDA)

In [16]:
# Lectura de los CSV en un df respectivamente

ruta = "csv_reviews.csv"
df_reviews_copy = pd.read_csv(ruta, low_memory=False)

In [17]:
# Ver dimensiones
df_reviews_copy.shape

(25799, 3)

In [18]:
# Revisar tipos de columnas y nulos
df_reviews_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [19]:
# Revisar estructura general
df_reviews_copy.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [20]:
# Conocer estructura de la columna "reviews"
data = df_reviews_copy.loc[0, "reviews"]
print("tipo dato: ", type(data),"\n")
print(data)

# Convertir la cadena a una lista de Python utilizando ast.literal_eval
lista_python = ast.literal_eval(data)

# Convertir la lista en un DataFrame utilizando pd.DataFrame()
df = pd.DataFrame(lista_python)
df.head(10)

tipo dato:  <class 'str'> 

[{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': "It's unique and worth a playthrough."}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]


,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [21]:
# convertir registro de tip str a tipo lista

def convertir_str_list(registro):
    '''
    Funcion para convertir cada registro de formato str a lista.
    Recibe como parámetro un str con formato lista
    '''
    list_registro = ast.literal_eval(registro)          # convertir cada registro str a lista
    return list_registro

In [22]:
# sacar copia a df y convertir registros a listas

df_reviews_copy_2 = df_reviews_copy.copy()
df_reviews_copy_2['reviews'] = df_reviews_copy_2['reviews'].apply(convertir_str_list) # para cada fila de la columna items ejecuta la funcion de agregar la cantidad de registros por lista

In [23]:
# desanidar columna "review"

list_complete_dict = []
for indice, fila in df_reviews_copy_2.iterrows():       # recorrer cada fila del dataframe
    user_id = fila['user_id']                           # guarda cada id de un usuario en una variable "user_id"
    user_url = fila['user_url']
    for dict_item_user in fila['reviews']:              # recorre cada fila de la columna "items"
        dict_item_user['user_id'] = user_id             # agrega una clave al diccionario que contiene el id del usuario
        dict_item_user['user_url'] = user_url
        list_complete_dict.append(dict_item_user)       # agrega cada dict a una lista general

df_reviews_copy_3 = pd.DataFrame(list_complete_dict)    # crea un dataframe con la lista de dict
orden_columnas = ['user_id', 'user_url', "funny", "posted", "last_edited", 'item_id', 'helpful', 'recommend', 'review']
df_reviews_copy_3 = df_reviews_copy_3[orden_columnas]   # organiza las columnas

In [24]:
df_reviews_copy_3.head(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [25]:
# Revisar los tipos de datos de cada columna y el porcentaje de nulos
verificar_tipo_datos(df_reviews_copy_3)

user_id  (nulos) =  0
user_url  (nulos) =  0
funny  (nulos) =  0
posted  (nulos) =  0
last_edited  (nulos) =  0
item_id  (nulos) =  0
helpful  (nulos) =  0
recommend  (nulos) =  0
review  (nulos) =  0

filas completamente nulas:  0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%
0,user_id,[<class 'str'>],100.0,0.0
1,user_url,[<class 'str'>],100.0,0.0
2,funny,[<class 'str'>],100.0,0.0
3,posted,[<class 'str'>],100.0,0.0
4,last_edited,[<class 'str'>],100.0,0.0
5,item_id,[<class 'str'>],100.0,0.0
6,helpful,[<class 'str'>],100.0,0.0
7,recommend,[<class 'bool'>],100.0,0.0
8,review,[<class 'str'>],100.0,0.0


In [26]:
df_reviews_copy_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


#### **1-2- csv_items.csv = australian_user_items.json**

- Dimensiones y tipos de variables

**Observaciones y conclusiones:**

- Se revisa dimensiones, cantidad de columnas, estructura general y no se encuentran filas completamente nulas.
- Dado que la columna "items" contiene una lista de diccionarios se procede a desanidar y dejar un dataset completo desanidado
- El dataset queda listo para exportar a CSV para realizar el "Analisis Exploratorio de Datos" (EDA)

In [27]:
# Lectura de los CSV en un df respectivamente

ruta = "csv_items.csv"
df_items_copy = pd.read_csv(ruta, low_memory=False)

In [28]:
# Ver dimensiones
df_items_copy.shape

(88310, 5)

In [29]:
# Revisar tipos de columnas y nulos
df_items_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [30]:
# Revisar estructura general
df_items_copy.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [31]:
# Conocer estructura de la columna "items"
data = df_items_copy.loc[0, "items"]
print("tipo dato: ", type(data),"\n")
print(data)

# Convertir la cadena a una lista de Python utilizando ast.literal_eval
lista_python = ast.literal_eval(data)

# Convertir la lista en un DataFrame utilizando pd.DataFrame()
df = pd.DataFrame(lista_python)
df.head(10)

tipo dato:  <class 'str'> 

[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 1853, 'pl

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
5,60,Ricochet,0,0
6,70,Half-Life,0,0
7,130,Half-Life: Blue Shift,0,0
8,300,Day of Defeat: Source,4733,0
9,240,Counter-Strike: Source,1853,0


In [32]:
# sacar copia a df y convertir registros a listas

df_items_copy_2 = df_items_copy.copy()
df_items_copy_2['items'] = df_items_copy_2['items'].apply(convertir_str_list) # para cada fila de la columna items ejecuta la funcion de agregar la cantidad de registros por lista

In [33]:
# desanidar columna "item"

list_complete_dict = []
for indice, fila in df_items_copy_2.iterrows():       # recorrer cada fila del dataframe
    user_id = fila['user_id']                           # guarda cada id de un usuario en una variable "user_id"
    items_count = fila['items_count']
    steam_id = fila['steam_id']
    user_url = fila['user_url']	
    for dict_item_user in fila['items']:              # recorre cada fila de la columna "items"
        dict_item_user['user_id'] = user_id             # agrega una clave al diccionario que contiene el id del usuario
        dict_item_user['items_count'] = items_count
        dict_item_user['steam_id'] = steam_id
        dict_item_user['user_url'] = user_url
        list_complete_dict.append(dict_item_user)       # agrega cada dict a una lista general

df_items_copy_3 = pd.DataFrame(list_complete_dict)    # crea un dataframe con la lista de dict
orden_columnas = ['user_id', "items_count", "steam_id",'user_url', "item_id", "item_name", "playtime_forever", "playtime_2weeks"]
df_items_copy_3 = df_items_copy_3[orden_columnas]   # organiza las columnas

In [34]:
df_items_copy_3.head(5)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [35]:
# Revisar los tipos de datos de cada columna y el porcentaje de nulos
verificar_tipo_datos(df_items_copy_3)

user_id  (nulos) =  0
items_count  (nulos) =  0
steam_id  (nulos) =  0
user_url  (nulos) =  0
item_id  (nulos) =  0
item_name  (nulos) =  0
playtime_forever  (nulos) =  0
playtime_2weeks  (nulos) =  0

filas completamente nulas:  0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%
0,user_id,[<class 'str'>],100.0,0.0
1,items_count,[<class 'int'>],100.0,0.0
2,steam_id,[<class 'int'>],100.0,0.0
3,user_url,[<class 'str'>],100.0,0.0
4,item_id,[<class 'str'>],100.0,0.0
5,item_name,[<class 'str'>],100.0,0.0
6,playtime_forever,[<class 'int'>],100.0,0.0
7,playtime_2weeks,[<class 'int'>],100.0,0.0


In [36]:
df_items_copy_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          int64 
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(4), object(4)
memory usage: 314.5+ MB


## **2- EXPORTAR A CSV**

**Observaciones y conclusiones:**

Se exporta cada dataset a un archivo CSV independiente de la siguiente forma:
  - Archivo "output_steam_games.json" = etl_games.csv
  - Archivo "australian_user_reviews.json" = etl_reviews.csv
  - Archivo "australian_users_items.json" = etl_items.csv

In [37]:
# Guardar el df en formato CSV para proceder al EDA
df_games_copy.to_csv('etl_games.csv', index=False)

In [38]:
# Guardar el df en formato CSV para proceder al EDA
df_reviews_copy_3.to_csv('etl_reviews.csv', index=False)

In [39]:
# Guardar el df en formato CSV para proceder al EDA
df_items_copy_3.to_csv('etl_items.csv', index=False)